In [13]:
from math_utils import *
import numpy as np
from scipy import linalg
from sktensor import ktensor

In [19]:
def gen_tensor(N,R):
    # all orthogonal cases
    U = np.random.random((N, R))
    U_orth = linalg.orth(U)

    V = np.random.random((N, R))
    V_orth = linalg.orth(V)

    W = np.random.random((N, R))
    W_orth = linalg.orth(W)

    Lambda = np.random.random((R,))

    X = ktensor([U_orth, V_orth, W_orth], lmbda=Lambda)
    X_ten = X.totensor()

    return X_ten

In [43]:
def gen_matrix(N,R):
    # all orthogonal cases
    U = np.random.random((N, R))
    U_orth = linalg.orth(U)

    V = np.random.random((N, R))
    V_orth = linalg.orth(V)

    Lambda = np.random.random((R,))


    X_mat = np.dot(np.dot(U_orth, np.diag(Lambda)),V_orth.T)

    return X_mat

In [55]:
def gen_mask(shape,obv_ratio):
    '''
    Return a mask matrix, 1 for observed, 0 for missing. 
    Args:
        obv_ratio: (0,1) observation ratio
        N: size of the mask
    '''
    obv_idx = np.random.choice([0, 1], size=shape, p=[1.0-obv_ratio, obv_ratio])
    Omega = np.reshape(obv_idx, shape)
    return Omega 

In [51]:
from cvxpy import *
from numpy import linalg as LA
def matrix_recovery(Omega,X, succ_thres):
    '''
    Return whether a matrix is recovered given a threshold
    Args:
        Omega: observed entires, true indicate observed
        X: ground truth 
        succ_thres: 
    Returns:
        recover decision under succ_thres
    '''
    n_rows, n_cols = X.shape
    X_opt = Variable(n_rows, n_cols)
     
    obj = Minimize(norm(X_opt, "nuc"))
    constraints = [mul_elemwise(Omega, X_opt) == mul_elemwise(Omega, X)]
    prob = Problem(obj, constraints)
    # Use SCS to solve the problem.
    prob.solve(verbose=True, solver=SCS) 
    err = LA.norm(X_opt.value-X, 'fro')/LA.norm(X, 'fro')
    print 'recovery err:', err
    if  err < succ_thres:
        return True
    else:
        return False

In [9]:
def tensor_recovery(Omega,X, succ_thres):
    '''
    Return whether an (orthogonal) tensor is recovered given a threshold
    Args:
        Omega: observed entires, true indicate observed
        X: ground truth 
        succ_thres: 
    Returns:
        recover decision under succ_thres
    '''
    n_rows, n_cols = X.shape
    X_opt = Variable(n_rows, n_cols)
     
        
    obj = Minimize(norm(X_opt, "nuc")) # tensor norm as algebraic mean of matrix norm
    constraints = [mul_elemwise(Omega, X_opt) == mul_elemwise(Omega, X)]
    prob = Problem(obj, constraints)
    # Use SCS to solve the problem.
    prob.solve(verbose=True, solver=SCS) 
    err = LA.norm(X_opt.value-X, 'fro')/LA.norm(X, 'fro')
    print 'recovery err:', err
    if  err < succ_thres:
        return True
    else:
        return False

In [ ]:
n_exps = 50;
n_rows = 20;
n_cols = 20;

R = 5; # rank
N = 20; # size

obv_ratio = 0.8;
succ_thres = 1e-3;

mat_recv_prob = np.zeros((n_rows, n_cols))
#for rank_ratio in linspace(0,1,n_rows):
X_mat = gen_matrix (N,R)
#    for obv_ratio in linspace(0,1,n_cols):

paralle n_exps:
Omega = gen_mask((N,N),obv_ratio)
if matrix_recovery(Omega, X, num_exp,succ_thres):
    
        #rec_prob_mat = tensor_recovery(Omega, X,num_exp,succ_thres)
        
    